In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import time
from tensorboard.plugins.hparams import api as hp

2024-01-03 14:27:51.246532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 14:27:51.611840: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 14:27:51.623700: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-03 14:27:51.623735: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### 1. Model Architecture

In [2]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.0, 0.15, 0.3, 0.45]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'tanh']))
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.0001, 0.001, 0.01, 0.1]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([0, 1, 2, 3, 4]))


In [3]:
# def createModel(config):
#   model = Sequential()
#   model.add(Dense(config[HP_NUM_UNITS], activation=config[HP_ACTIVATION], input_shape=(512,)))
#   model.add(Dropout(config[HP_DROPOUT]))

#   for i in range(config[HP_NUM_LAYERS]):
#     model.add(Dense(config[HP_NUM_UNITS], activation=config[HP_ACTIVATION]))
#     # model.add(Dropout(config[HP_DROPOUT]))

#   model.add(Dense(1, activation='sigmoid'))

#   if config[HP_OPTIMIZER] == 'adam':
#     optimizer = Adam(learning_rate=config[HP_LR])
#   elif config[HP_OPTIMIZER] == 'sgd':
#     optimizer = SGD(learning_rate=config[HP_LR])
#   else:
#     optimizer = RMSprop(learning_rate=config[HP_LR])

#   model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#   return model

def createModel(config):
  model = Sequential()  

  dropoutVal = 0.5

  # CNN models
  model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(7, 7, 1280)))
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropoutVal))

  model.add(Conv2D(64, (3, 3), activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropoutVal))

  model.add(Conv2D(256, (3, 3), activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(dropoutVal))

  model.add(Flatten())
  model.add(Dense(512, activation="relu"))
  model.add(Dense(32, activation="relu"))
  model.add(Dense(1, activation="sigmoid"))


  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

### 2. Training 

In [4]:
# Euclidean distance 
def linear_diff(np1, np2):
  return np1 - np2

# TODO - also try absolute difference, cosine similarity, etc.

In [5]:
features = np.load("./train_X.npy")
trainY = np.load("./train_y.npy")

print(features.shape)
trainX = []
for i in range(0, len(features)):
  trainX.append(linear_diff(features[i][0], features[i][1]))

trainX = np.array(trainX)
print(trainX.shape)
print(trainY.shape)


(13032, 2, 7, 7, 1280)
(13032, 7, 7, 1280)
(13032,)


In [6]:
np.save("./train_X_diff.npy", trainX)
np.save("./train_y_diff.npy", trainY)

In [5]:
features_val = np.load("./val_X.npy")
valY = np.load("./val_y.npy")

valX = []
for i in range(0, len(features_val)):
  valX.append(linear_diff(features_val[i][0], features_val[i][1]))

valX = np.array(valX)
print(valX.shape)


(2592, 7, 7, 1280)


In [6]:
np.save("./val_X_diff.npy", valX) 
np.save("./val_y_diff.npy", valY)

In [5]:
# Load the diff version
trainX = np.load("./train_X_diff.npy")
trainY = np.load("./train_y_diff.npy")
valX = np.load("./val_X_diff.npy")
valY = np.load("./val_y_diff.npy")

In [12]:
trainX.shape, trainY.shape, valX.shape, valY.shape

((5792, 512), (5792,), (1152, 512), (1152,))

In [7]:
def stringifyConfig(config):
  return f"numLayers-{config[HP_NUM_LAYERS]}_numUnits-{config[HP_NUM_UNITS]}_dropout-{config[HP_DROPOUT]}_optimizer-{config[HP_OPTIMIZER]}_activation-{config[HP_ACTIVATION]}_lr-{config[HP_LR]}"

In [8]:
method = "flatten&eudlidian"

for NUM_UNIT in HP_NUM_UNITS.domain.values:
  for NUM_LAYER in HP_NUM_LAYERS.domain.values:

    # config = {
    #   HP_NUM_UNITS: NUM_UNIT,
    #   HP_DROPOUT: 0,
    #   HP_OPTIMIZER: 'adam',
    #   HP_ACTIVATION: 'relu',
    #   HP_LR: 0.001,
    #   HP_NUM_LAYERS: NUM_LAYER
    # }

    config = {
      HP_NUM_UNITS: 256,
      HP_DROPOUT: 0.5,
      HP_OPTIMIZER: 'adam',
      HP_ACTIVATION: 'relu',
      HP_LR: 0.001,
      HP_NUM_LAYERS: 2
    }

    # Callbacks
    modelID = int(time.time())
    modelName = stringifyConfig(config) + str(modelID)

    print("\nmodelID: ", modelID)

    # Callbacks
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=f"./logs/{method}/{modelName}", histogram_freq=1)

    checkpoint = ModelCheckpoint("./models/" + method + "/" + modelName + ".h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

    hparms = hp.KerasCallback("./logs/" + method + "/" + modelName, config)

    callbacks_list = [checkpoint, early, tensorboard_callback, hparms]

    model = createModel(config)

    print("Parameters: ", model.count_params())
    print("HP_NUM_UNITS: ", NUM_UNIT)
    print("HP_NUM_LAYERS: ", NUM_LAYER)


    model.fit(trainX, trainY, batch_size=32, epochs=100, callbacks=callbacks_list, shuffle=True, verbose=1, validation_data=(valX, valY))

    break
  break


modelID:  1704313741


2024-01-03 14:29:01.502107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-03 14:29:01.502142: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-03 14:29:01.502206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (blankk): /proc/driver/nvidia/version does not exist
2024-01-03 14:29:01.503291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Parameters:  682913
HP_NUM_UNITS:  16
HP_NUM_LAYERS:  0
Epoch 1/100
408/408 [==============================] - ETA: 0s - loss: 0.4832 - accuracy: 0.7405
Epoch 1: val_loss improved from inf to 0.52177, saving model to ./models/flatten&eudlidian/numLayers-2_numUnits-256_dropout-0.5_optimizer-adam_activation-relu_lr-0.0011704313741.h5
408/408 [==============================] - 26s 55ms/step - loss: 0.4832 - accuracy: 0.7405 - val_loss: 0.5218 - val_accuracy: 0.7346
Epoch 2/100
407/408 [============================>.] - ETA: 0s - loss: 0.2330 - accuracy: 0.9192
Epoch 2: val_loss improved from 0.52177 to 0.40762, saving model to ./models/flatten&eudlidian/numLayers-2_numUnits-256_dropout-0.5_optimizer-adam_activation-relu_lr-0.0011704313741.h5
408/408 [==============================] - 21s 52ms/step - loss: 0.2334 - accuracy: 0.9191 - val_loss: 0.4076 - val_accuracy: 0.7998
Epoch 3/100
408/408 [==============================] - ETA: 0s - loss: 0.1635 - accuracy: 0.9438
Epoch 3: val_loss imp

### Testing

In [14]:
# Load best model
model = keras.models.load_model("./models/" + method + "/" + modelName + ".h5")

In [12]:
textY = np.load("./test_y.npy") 
textX = []
features = np.load("./test_X.npy")
for i in range(0, len(features)):
  textX.append(linear_diff(features[i][0], features[i][1]))

textX = np.array(textX)

print(textX.shape)
print(textY.shape)



(2916, 7, 7, 1280)
(2916,)


In [15]:
score = model.evaluate(textX, textY)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


92/92 [==============================] - 2s 20ms/step - loss: 0.4715 - accuracy: 0.7850
Test loss: 0.47152179479599
Test accuracy: 0.7849794030189514


(5792, 512)

array([ 8.94228   ,  4.2660823 , 11.17209   , 23.513826  ,  2.9565973 ,
        6.5381765 ,  0.        ,  5.5042477 ,  0.        ,  1.1767998 ,
        7.3116856 ,  0.        ,  2.3247094 ,  0.        ,  5.8922615 ,
       17.017056  , 26.68887   ,  5.6589365 ,  1.4295353 ,  6.4644775 ,
        4.9205627 ,  7.2869186 , 23.883442  , 18.176207  , 15.436508  ,
       30.406403  ,  8.54992   , 33.56736   ,  1.7889771 ,  7.728585  ,
        1.51395   , 23.54344   , 17.904615  ,  3.665018  , 13.18933   ,
       28.25327   , 18.471085  ,  0.39198163,  3.855914  ,  0.        ,
       32.565075  ,  1.9633389 ,  6.242937  , 91.320526  , 11.824903  ,
       44.20205   ,  7.252417  , 28.722588  ,  4.6957383 ,  3.3829384 ,
       11.56041   ,  0.        , 24.655628  , 17.028023  ,  0.99671066,
       62.72991   ,  7.6214714 , 25.34861   , 11.781673  , 10.941526  ,
        0.        ,  3.4047937 ,  3.5542145 , 18.49507   ,  3.4157085 ,
       25.44946   , 20.102911  ,  0.        ,  6.6191225 ,  1.71